In [32]:
# require Python 3.12: using `type` soft keyword

from pprint import pprint
from resources import (
    CodeableConcept,
    Coding,
    Condition,
    dateType,
    Encounter,
    HumanName,
    Identifier,
    List_,
    Literal_,
    Observation,
    Organization,
    Patient,
    Procedure,
    Reference,
    Questionnaire,
    QuestionnaireResponse,
)

In [33]:
# resources are FHIR R4B resources, with all fields are optional
patient_base = Patient()
patient_base.__dict__

{'resourceType': 'Patient',
 'id': None,
 'id__ext': None,
 'meta': None,
 'text': None,
 'name': None,
 'link': None,
 'photo': None,
 'active': None,
 'active__ext': None,
 'gender': None,
 'gender__ext': None,
 'telecom': None,
 'address': None,
 'contact': None,
 'language': None,
 'language__ext': None,
 'contained': None,
 'extension': None,
 'birthDate': None,
 'birthDate__ext': None,
 'identifier': None,
 'deceasedBoolean': None,
 'deceasedBoolean__ext': None,
 'deceasedDateTime': None,
 'deceasedDateTime__ext': None,
 'implicitRules': None,
 'implicitRules__ext': None,
 'maritalStatus': None,
 'communication': None,
 'multipleBirthBoolean': None,
 'multipleBirthBoolean__ext': None,
 'multipleBirthInteger': None,
 'multipleBirthInteger__ext': None,
 'modifierExtension': None,
 'generalPractitioner': None,
 'managingOrganization': None}

In [34]:
# IPS has put constrained on Patient resource, some fields are now mandatory
class PatientIPS(Patient):
    name: List_[HumanName] = None
    birthdate: dateType = None


# note given name is a list, and there is only one family name
patient_IPS = PatientIPS(
    name=[HumanName(given=["Daniel"], family="Kapitan")], birthdate="09-09-1973"
)
for k, v in patient_IPS.__dict__.items():
    if v:
        pprint(f"{k}: {getattr(patient_IPS, k)}")

'resourceType: Patient'
('name: [HumanName(id=None, id__ext=None, use=None, use__ext=None, text=None, '
 "text__ext=None, given=['Daniel'], given__ext=None, family='Kapitan', "
 'family__ext=None, prefix=None, prefix__ext=None, suffix=None, '
 'suffix__ext=None, period=None, extension=None)]')
'birthdate: 09-09-1973'


In [35]:
codeEDD = CodeableConcept(
    coding=[
        Coding(
            code="11778-8",
            system="https://loinc.org",
            display="Delivery date Estimated",
        )
    ]
)

In [36]:
Observation(code=codeEDD)

ValidationError: 1 validation error for Observation
status
  Field required [type=missing, input_value={'code': CodeableConcept(...None)], extension=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [37]:
Observation(
    code=codeEDD,
    subject=Reference(
        reference="https://some.fhir/server/dkapitan",
    ),
)

ValidationError: 1 validation error for Observation
status
  Field required [type=missing, input_value={'code': CodeableConcept(...=None, identifier=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [26]:
type EddMethod = Literal_[
    Coding(
        code="11778-8", system="https://loinc.org", display="Delivery date Estimated"
    ),
    Coding(
        code="11778-6",
        system="https://loinc.org",
        display="Delivery date Estimated from last mentrual period",
    ),
    Coding(
        code="11778-4",
        system="https://loinc.org",
        display="Delivery date Estimated from ovulation date",
    ),
]

In [25]:
# Observation
# fixed code list how EDD is estimated using LOINC codes
# https://build.fhir.org/ig/HL7/fhir-ips/ValueSet-edd-method-uv-ips.html
# note that ObservationPregnancyEddIPS removes bodysite, method, specimen, device, referenceRange and component
# we haven't implemented that here
# Main point that we demonstrate is value binding
class ObservationPregnancyEddIPS(Observation):
    code: CodeableConcept(coding=List_[EddMethod])
    subject: str


some_edd = ObservationPregnancyEddIPS(
    code=CodeableConcept(
        coding=[
            Coding(
                code="11778-8",
                system="https://loinc.org",
                display="Delivery date Estimated",
            )
        ]
    ),
    subject=Reference(
        reference="https://some.fhir/server/dkapitan",
    ),
)

ValidationError: 1 validation error for CodeableConcept
coding.0
  Input should be a valid dictionary or instance of Coding [type=model_type, input_value=typing.Unpack[typing.List[EddMethod]], input_type=_UnpackGenericAlias]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [30]:
isinstance(codeEDD, EddMethod)

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union